In [246]:
# ===== CONFIG =====
import os, re, glob
import numpy as np
import pandas as pd
from sklearn.linear_model import LinearRegression

# Only use the *_new.csv files
DATA_GLOB = "../data/Raw/savant_data_*_new.csv"
OUTPUT_CSV = "../data/Derived/BPI_watchlist.csv"

# Analysis switches
MIN_PA = 100
ROLLING_YEARS = None      # e.g., 2 -> last 2 seasons per player; None = all seasons
USE_R2_WEIGHTING = True   # multiply each z by its own per-stat R²
TOP_N = 25
WEIGHT_PRESET = "spec_default"  # "spec_default" or "expanded"

# ID/Name candidates (we’ll pick what exists)
ID_CANDIDATES = ["player_id", "batter", "mlbam_id"]
NAME_CANDIDATES = ["player_name", "name"]

# Canonical stat list (auto-skip if missing)
CORE_STATS = [
    "launch_speed","hardhit_percent","barrels_per_bbe_percent","bbdist",
    "swing_miss_percent","bb_percent","k_percent",
    "launch_angle","attack_angle","rate_ideal_attack_angle",
    "xwoba","xba","xslg","wobadiff","xbadiff","xslgdiff",
]
EXPANDED_STATS = [
    "barrels_per_pa_percent","bat_speed","adj_exit_velocity","xobp","obp_diff","iso"
]
ALL_STATS = CORE_STATS + EXPANDED_STATS

# Which metrics are “lower is better”: flip their slopes so higher = better
SIGN_FLIP = {"swing_miss_percent", "k_percent"}

# Weight presets (we skip missing features automatically)
WEIGHTS_SPEC_DEFAULT = {
    "launch_speed_z": 0.18, "hardhit_percent_z": 0.14, "barrels_per_bbe_percent_z": 0.14, "bbdist_z": 0.10,
    "swing_miss_percent_z": 0.10, "bb_percent_z": 0.06, "k_percent_z": 0.04,
    "launch_angle_z": 0.06, "attack_angle_z": 0.04, "rate_ideal_attack_angle_z": 0.02,
    "xwoba_z": 0.04, "xba_z": 0.02, "xslg_z": 0.02, "wobadiff_z": 0.02, "xbadiff_z": 0.01, "xslgdiff_z": 0.01,
}
WEIGHTS_EXPANDED = {
    **WEIGHTS_SPEC_DEFAULT,
    "barrels_per_pa_percent_z": 0.06, "bat_speed_z": 0.06, "adj_exit_velocity_z": 0.04,
    "xobp_z": 0.02, "obp_diff_z": 0.01, "iso_z": 0.02,
}
def get_weights():
    return WEIGHTS_EXPANDED if WEIGHT_PRESET == "expanded" else WEIGHTS_SPEC_DEFAULT

# Aliases: map our canonical names to possible CSV variants
ALIASES = {
    "launch_speed": ["launch_speed","exit_velocity","avg_ev","exit_velocity_avg"],
    "hardhit_percent": ["hardhit_percent","hard_hit_percent","hard_hit_pct"],
    "barrels_per_bbe_percent": ["barrels_per_bbe_percent","barrel_bbe_percent","barrel_bbe_pct"],
    "barrels_per_pa_percent": ["barrels_per_pa_percent","barrel_pa_percent","barrel_pa_pct"],
    "bbdist": ["bbdist","hit_distance","avg_distance"],
    "swing_miss_percent": ["swing_miss_percent","whiff_percent","whiff_rate"],
    "bb_percent": ["bb_percent","walk_percent","bb_pct"],
    "k_percent": ["k_percent","strikeout_percent","k_pct"],
    "launch_angle": ["launch_angle","avg_la"],
    "attack_angle": ["attack_angle"],
    "rate_ideal_attack_angle": ["rate_ideal_attack_angle","ideal_attack_angle_percent","ideal_attack_angle_%"],
    "xwoba": ["xwoba"], "xba": ["xba","xBA","x_ba"], "xslg": ["xslg","xSLG","x_slg"],
    "wobadiff": ["wobadiff","woba_minus_xwoba","woba_diff"],
    "xbadiff": ["xbadiff","ba_minus_xba","ba_diff"],
    "xslgdiff": ["xslgdiff","slg_minus_xslg","slg_diff"],
    "xobp": ["xobp","xOBP","x_obp"], "obp_diff": ["obp_diff","obp_minus_xobp"],
    "iso": ["iso"], "bat_speed": ["bat_speed"], "adj_exit_velocity": ["adj_exit_velocity","adjusted_exit_velocity"],
    "pa": ["pa","PA"],
}

def coalesce(df, canonical):
    for cand in ALIASES.get(canonical, [canonical]):
        if cand in df.columns:
            return cand
    return None


In [248]:
# ===== LOAD & MERGE (only *_new.csv) =====
paths = sorted(glob.glob(DATA_GLOB))
if not paths:
    raise FileNotFoundError(f"No files matched {DATA_GLOB}")

print("Using files:")
for p in paths:
    print(" -", os.path.basename(p))

frames = []
for p in paths:
    m = re.search(r"(19|20)\d{2}", os.path.basename(p))
    if not m:
        raise ValueError(f"Cannot infer season from filename: {p}")
    yr = int(m.group(0))
    df = pd.read_csv(p)
    df["season"] = yr
    frames.append(df)

raw = pd.concat(frames, ignore_index=True)

# choose id and name cols
id_col = next((c for c in ID_CANDIDATES if c in raw.columns), None)
name_col = next((c for c in NAME_CANDIDATES if c in raw.columns), None)
if name_col is None:
    raw["player_name"] = raw[id_col].astype(str) if id_col else np.arange(len(raw)).astype(str)
    name_col = "player_name"

# filter by PA
pa_col = coalesce(raw, "pa")
if pa_col is None:
    raise ValueError("No PA column found (tried: 'pa', 'PA').")
raw = raw[raw[pa_col] >= MIN_PA].copy()

raw = raw.sort_values([name_col, "season"]).reset_index(drop=True)
print("\nSeasons present after PA filter:", sorted(raw["season"].unique()))
print("Rows after PA filter:", len(raw))
print("ID:", id_col, "| Name:", name_col, "| PA:", pa_col)


Using files:
 - savant_data_2021_new.csv
 - savant_data_2022_new.csv
 - savant_data_2023_new.csv
 - savant_data_2025_new.csv

Seasons present after PA filter: [2021, 2022, 2023, 2025]
Rows after PA filter: 1616
ID: player_id | Name: player_name | PA: pa


In [250]:
# ===== SLOPES & R² =====

def fit_slope(x_years, y_vals):
    x = np.asarray(x_years, dtype=float).reshape(-1,1)
    y = np.asarray(y_vals, dtype=float)
    if len(np.unique(x)) < 2 or len(y) < 2:
        return np.nan, np.nan
    m = LinearRegression().fit(x, y)
    return float(m.coef_[0]), float(m.score(x, y))

def compute_slopes(df, stats, id_col, name_col, rolling_years=None):
    key = id_col if id_col else name_col
    out_rows = []
    for pid, g in df.groupby(key, sort=False):
        gg = g.sort_values("season")
        seasons = gg["season"].unique()
        if rolling_years is not None and len(seasons) > 0:
            max_s = seasons.max()
            gg = gg[gg["season"].between(max_s - rolling_years + 1, max_s)]
            seasons = gg["season"].unique()
        if len(seasons) < 2:
            continue

        row = {key: pid, "player_name": gg.iloc[-1][name_col] if name_col in gg.columns else str(pid)}
        for met in stats:
            col = coalesce(gg, met)
            if not col:
                continue
            tmp = gg[["season", col]].dropna().groupby("season", as_index=False)[col].mean()
            if len(tmp) < 2:
                continue
            slope, r2 = fit_slope(tmp["season"].values, tmp[col].values)
            row[f"{met}_slope"] = slope
            row[f"{met}_r2"] = r2
        out_rows.append(row)

    slopes = pd.DataFrame(out_rows)
    # if grouping key isn't name, make sure we carry name along
    if key != "player_name" and "player_name" not in slopes.columns and name_col in df.columns:
        nm = df[[key, name_col]].drop_duplicates(subset=[key], keep="last")
        slopes = slopes.merge(nm, on=key, how="left")
    return slopes

slopes_df = compute_slopes(raw, ALL_STATS, id_col=id_col, name_col=name_col, rolling_years=ROLLING_YEARS)
print("Slopes shape:", slopes_df.shape)
slopes_df.head(3)


Slopes shape: (462, 42)


,player_id,player_name,launch_speed_slope,launch_speed_r2,hardhit_percent_slope,hardhit_percent_r2,barrels_per_bbe_percent_slope,barrels_per_bbe_percent_r2,bbdist_slope,bbdist_r2,...,xslgdiff_slope,xslgdiff_r2,barrels_per_pa_percent_slope,barrels_per_pa_percent_r2,bat_speed_slope,bat_speed_r2,xobp_slope,xobp_r2,iso_slope,iso_r2
0,682928,"Abrams, CJ",1.285714,0.824176,3.118784,0.855605,1.779113,0.497771,8.500000,0.662844,...,0.001214,0.793956,1.548858,0.487716,0.45,1.0,-0.000643,0.008242,0.034643,0.498244
1,547989,"Abreu, José",-1.050000,0.679158,-2.601133,0.437535,1.358177,0.572328,8.000000,0.884793,...,-0.021000,0.417087,1.837598,0.797763,NaN,NaN,-0.039500,0.521765,0.008500,0.028153
2,660670,"Acuña Jr., Ronald",0.074286,0.004426,-0.084534,0.002032,-0.861083,0.216592,-1.057143,0.035080,...,-0.003086,0.035880,0.178038,0.016352,-0.40,1.0,-0.022943,0.696289,-0.005429,0.016478


In [237]:
# ===== Z-SCORES =====

# 1) flip “lower-is-better” slopes so higher = better
for met in SIGN_FLIP:
    c = f"{met}_slope"
    if c in slopes_df.columns:
        slopes_df[c] = -slopes_df[c]

# 2) collect slope cols
slope_cols = [c for c in slopes_df.columns if c.endswith("_slope")]

# 3) z-score each slope column robustly
Z = pd.DataFrame(index=slopes_df.index)
for c in slope_cols:
    v = pd.to_numeric(slopes_df[c], errors="coerce").replace([np.inf, -np.inf], np.nan)
    if v.notna().sum() < 2:
        Z[c.replace("_slope", "_z")] = 0.0
        continue
    mu, sd = v.mean(skipna=True), v.std(ddof=0, skipna=True)
    Z[c.replace("_slope", "_z")] = ((v - mu) / (sd if sd and not np.isclose(sd,0) else 1.0)).fillna(0.0)

# 4) R² weighting (optional)
if USE_R2_WEIGHTING:
    for zc in list(Z.columns):
        r2c = zc.replace("_z", "_r2")
        if r2c in slopes_df.columns:
            Z[zc] = Z[zc] * slopes_df[r2c].fillna(0.0)

# Assemble working frame with names
keys = [c for c in [id_col, "player_name"] if c and c in slopes_df.columns]
work = pd.concat([slopes_df[keys].reset_index(drop=True), Z.reset_index(drop=True)], axis=1)
print("Z frame:", work.shape)
work.head(3)


Z frame: (585, 22)


,player_id,player_name,launch_speed_z,hardhit_percent_z,barrels_per_bbe_percent_z,bbdist_z,swing_miss_percent_z,bb_percent_z,k_percent_z,launch_angle_z,...,xwoba_z,xba_z,xslg_z,wobadiff_z,xbadiff_z,xslgdiff_z,barrels_per_pa_percent_z,bat_speed_z,xobp_z,iso_z
0,682928,"Abrams, CJ",0.697018,0.677702,0.478156,0.892977,-0.067032,0.227111,-0.025198,0.483772,...,0.668931,0.193531,0.529132,0.371339,0.017553,0.213942,0.415339,0.054590,0.333636,0.591117
1,547989,"Abreu, José",-1.147654,-1.114072,-0.515842,0.000073,-0.090666,-0.550218,-0.069533,-0.077818,...,-1.317388,-0.779198,-0.849826,-0.381730,-0.598396,-0.165000,-0.371770,-1.897688,-1.546845,-0.528073
2,677800,"Abreu, Wilyer",-0.306602,-1.218719,0.998882,1.465235,3.033578,-2.002466,1.717334,1.961473,...,1.543290,1.771328,2.491998,-1.489435,-1.448763,-1.774169,1.855790,0.465419,0.060943,1.194572


In [252]:
# ===== BPI =====

def compute_bpi(df_z, weights: dict):
    z_cols = [c for c in df_z.columns if c.endswith("_z")]
    used = {k: v for k, v in weights.items() if k in z_cols}
    if not used:
        raise ValueError("No weighted features present.")
    n = len(df_z)
    wsum = np.zeros(n)
    feat_count = np.zeros(n, dtype=int)

    for k, w in used.items():
        vals = df_z[k].fillna(0.0).values
        wsum += vals * float(w)
        feat_count += df_z[k].notna().astype(int).values

    bpi = pd.Series(wsum, index=df_z.index, name="BPI")
    frac = feat_count / max(1, len(used))
    bpi_adj = bpi * frac
    mu, sd = bpi_adj.mean(), bpi_adj.std(ddof=0)
    bpi_scaled = 100 + 15 * ((bpi_adj - mu) / (sd if sd and not np.isclose(sd,0) else 1.0))

    out = df_z.copy()
    out["BPI"] = bpi
    out["BPI_adj"] = bpi_adj
    out["BPI_scaled"] = bpi_scaled
    return out, used

weights = get_weights()
watch_df, used_weights = compute_bpi(work, weights)

# Build a human-friendly view that ALWAYS includes player_name (falls back to ID if needed)
display_cols = []
if "player_name" in watch_df.columns:
    display_cols.append("player_name")
if id_col and id_col in watch_df.columns:
    display_cols.append(id_col)
display_cols += ["BPI","BPI_adj","BPI_scaled"]

watch_view = watch_df[display_cols].sort_values("BPI_adj", ascending=False).reset_index(drop=True)
print(f"Features used ({len(used_weights)}):", ", ".join(sorted(used_weights.keys())))
watch_view.head(10)


Features used (16): attack_angle_z, barrels_per_bbe_percent_z, bb_percent_z, bbdist_z, hardhit_percent_z, k_percent_z, launch_angle_z, launch_speed_z, rate_ideal_attack_angle_z, swing_miss_percent_z, wobadiff_z, xba_z, xbadiff_z, xslg_z, xslgdiff_z, xwoba_z


,player_name,player_id,BPI,BPI_adj,BPI_scaled
0,"Marcano, Tucupita",672779,2.469155,2.469155,172.605335
1,"Caminero, Junior",691406,1.766786,1.766786,152.050860
2,"Loftin, Nick",679845,1.728002,1.728002,150.915866
3,"Cordero, Franchy",614173,1.726558,1.726558,150.873616
4,"Barger, Addison",680718,1.719546,1.719546,150.668412
5,"Stowers, Kyle",669065,1.713853,1.713853,150.501798
6,"Rice, Ben",700250,1.544952,1.544952,145.559011
7,"Busch, Michael",683737,1.338921,1.338921,139.529616
8,"Alexander, Blaze",677942,1.301762,1.301762,138.442180
9,"Keith, Colt",690993,1.250164,1.250164,136.932181


In [254]:
# ===== OUTPUTS =====
top25 = watch_view.head(TOP_N).copy()
bot25 = watch_view.tail(TOP_N).copy() if len(watch_view) >= TOP_N else watch_view.iloc[::-1].head(TOP_N).copy()

print("\n=== TOP 25 (highest BPI_adj) ===")
display(top25)

print("\n=== BOTTOM 25 (lowest BPI_adj) ===")
display(bot25)

# Save full table (with names)
os.makedirs(os.path.dirname(OUTPUT_CSV), exist_ok=True)
meta = {
    "min_pa": MIN_PA,
    "rolling_years": ROLLING_YEARS,
    "use_r2_weighting": USE_R2_WEIGHTING,
    "weight_preset": WEIGHT_PRESET,
    "features_used": sorted(list(used_weights.keys())),
}
export = watch_view.copy()
for k, v in meta.items():
    export[k] = str(v)
export.to_csv(OUTPUT_CSV, index=False)
print(f"\nSaved: {OUTPUT_CSV}")



=== TOP 25 (highest BPI_adj) ===


,player_name,player_id,BPI,BPI_adj,BPI_scaled
0,"Marcano, Tucupita",672779,2.469155,2.469155,172.605335
1,"Caminero, Junior",691406,1.766786,1.766786,152.050860
2,"Loftin, Nick",679845,1.728002,1.728002,150.915866
3,"Cordero, Franchy",614173,1.726558,1.726558,150.873616
4,"Barger, Addison",680718,1.719546,1.719546,150.668412
5,"Stowers, Kyle",669065,1.713853,1.713853,150.501798
6,"Rice, Ben",700250,1.544952,1.544952,145.559011
7,"Busch, Michael",683737,1.338921,1.338921,139.529616
8,"Alexander, Blaze",677942,1.301762,1.301762,138.442180
9,"Keith, Colt",690993,1.250164,1.250164,136.932181



=== BOTTOM 25 (lowest BPI_adj) ===


,player_name,player_id,BPI,BPI_adj,BPI_scaled
560,"Díaz, Lewin",650331,-1.019118,-1.019118,70.522774
561,"Votto, Joey",458015,-1.031015,-1.031015,70.174618
562,"Herrera, Odúbel",546318,-1.035639,-1.035639,70.039312
563,"Collins, Zack",641470,-1.059805,-1.059805,69.332089
564,"Sweeney, Trey",700242,-1.088077,-1.088077,68.504741
565,"Noel, Jhonkensy",678877,-1.104705,-1.104705,68.018135
566,"Dalbec, Bobby",666915,-1.105068,-1.105068,68.007490
567,"Tena, José",677588,-1.106484,-1.106484,67.966056
568,"Fitzgerald, Tyler",666149,-1.134982,-1.134982,67.132083
569,"Adrianza, Ehire",501303,-1.193886,-1.193886,65.408293



Saved: ../data/Derived/BPI_watchlist.csv


In [258]:
import glob, os, re, pandas as pd

DATA_GLOBS = [
    "../data/Raw/savant_data_*.csv",
    "../data/Raw/savant_data_*_new.csv",
]

paths = sorted({p for pat in DATA_GLOBS for p in glob.glob(pat)})
print("Loaded files:")
for p in paths:
    yr = re.search(r"(19|20)\d{2}", os.path.basename(p))
    print(f" - {os.path.basename(p)}  | season parsed: {yr.group(0) if yr else '??'}")

# After you've run the merge step (raw DataFrame exists):
try:
    pa_col = next((c for c in ["pa","PA"] if c in raw.columns), None)
    print("\nSeasons present after PA filter:", sorted(raw["season"].unique()))
    print("Rows after PA filter:", len(raw))
    if pa_col:
        print(f"PA column: {pa_col} | MIN_PA threshold in config: {MIN_PA}")
except NameError:
    print("\nRun your load/merge cell first so 'raw' exists.")


Loaded files:
 - savant_data_2021.csv  | season parsed: 2021
 - savant_data_2021_new.csv  | season parsed: 2021
 - savant_data_2022.csv  | season parsed: 2022
 - savant_data_2022_new.csv  | season parsed: 2022
 - savant_data_2023.csv  | season parsed: 2023
 - savant_data_2023_new.csv  | season parsed: 2023
 - savant_data_2024.csv  | season parsed: 2024
 - savant_data_2025.csv  | season parsed: 2025
 - savant_data_2025_new.csv  | season parsed: 2025

Seasons present after PA filter: [2021, 2022, 2023, 2025]
Rows after PA filter: 1616
PA column: pa | MIN_PA threshold in config: 100
